In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from torchvision import transforms

SEED = 1360686
VAL_SPLIT = 0.1
BATCH_SIZE = 128
POLY_FEAT_N = 1000
TARGET_COLUMNS = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set the seed for reproducibility
np.random.seed(SEED)
torch.manual_seed(SEED)

# Read the data
train_data = pd.read_csv('./data/train.csv')
train_data['file_path'] = train_data['id'].apply(lambda x: os.path.join('./data/train_images', f"{x}.jpeg"))
test_data = pd.read_csv('./data/test.csv')
test_data['file_path'] = test_data['id'].apply(lambda x: os.path.join('./data/test_images', f"{x}.jpeg"))

# Split the data into training and validation sets
train_df, val_df = train_test_split(train_data, test_size=VAL_SPLIT, random_state=SEED)

# Standardize numerical features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_df.drop(columns=TARGET_COLUMNS + ['id', 'file_path']).values.astype(np.float32))
val_features = scaler.transform(val_df.drop(columns=TARGET_COLUMNS + ['id', 'file_path']).values.astype(np.float32))
test_features = scaler.transform(test_data.drop(columns=['id', 'file_path']).values.astype(np.float32))

# Load the pre-trained DINOv2 model
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg').to(device)
model.eval()

# Define the preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(224, interpolation=3),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Extract image embeddings
def get_image_embeddings_dino(model, preprocess, batch_size, df):
    image_embeddings = []
    for i in tqdm(range(0, len(df), batch_size)):
        paths = df['file_path'][i:i + batch_size]
        image_tensor = torch.stack([preprocess(Image.open(path)) for path in paths]).to(device)
        with torch.no_grad():
            curr_image_embeddings = model(image_tensor)
        image_embeddings.extend(curr_image_embeddings.cpu().numpy())
    return image_embeddings

train_image_embeddings = get_image_embeddings_dino(model, preprocess, BATCH_SIZE, train_df)
val_image_embeddings = get_image_embeddings_dino(model, preprocess, BATCH_SIZE, val_df)
test_image_embeddings = get_image_embeddings_dino(model, preprocess, BATCH_SIZE, test_data)

# Combine polynomial features with image features
poly = PolynomialFeatures(degree=2, include_bias=False)
train_poly_features = poly.fit_transform(train_features)[:, :POLY_FEAT_N]
val_poly_features = poly.transform(val_features)[:, :POLY_FEAT_N]
test_poly_features = poly.transform(test_features)[:, :POLY_FEAT_N]

train_combined_features = np.concatenate((train_poly_features, np.array(train_image_embeddings)), axis=1)
val_combined_features = np.concatenate((val_poly_features, np.array(val_image_embeddings)), axis=1)
test_combined_features = np.concatenate((test_poly_features, np.array(test_image_embeddings)), axis=1)

# Save data
train_combined_df.to_csv('./dinov2/run2/train_features_combined.csv', index=False)
val_combined_df.to_csv('./dinov2/run2/val_features_combined.csv', index=False)
test_combined_df.to_csv('./dinov2/run2/test_features_combined.csv', index=False)
train_df.to_csv('./dinov2/run2/train_splitted.csv', index=False)
val_df.to_csv('./dinov2/run2/val_splitted.csv', index=False)
test_data.to_csv('./dinov2/run2/test_splitted.csv', index=False)

Using cache found in /home/q258wu/.cache/torch/hub/facebookresearch_dinov2_main
/home/q258wu/miniconda3/envs/cs480p1/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/q258wu/miniconda3/envs/cs480p1/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/home/q258wu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/q258wu/.cache/torch/hub/fac